In [1]:
# %%bash
import os
os.environ['http_proxy'] = 'http://localhost:8081'
os.environ['https_proxy'] = 'http://localhost:8081'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'

/mnt/nfs-data/users/anhvth5/projects/guidance


In [3]:
from speedy import *
from guidance import *

In [4]:
data = load_by_ext('../tvpl_crawler/data/tvpl_qa_short_answer_giaothong/qa_short_answer/*')
questions = []
for qa in data:
    questions.extend(qa['questions'])

2023-12-30 17:36:19.396 | INFO     | speedy.utils:multi_process:262 - Multi processing load_by_ext | Num samples: 6788


  0%|          | 0/6788 [00:00<?, ?it/s]

In [5]:
# llm = models.get_qwen_guidance('../kilm-finetuning/output/Qwen-14B-Chat_on_giaothong_rag/', device_map=0,
                               # do_update_lm_head=True, echo=True)
# llm = models.get_qwen_guidance('../kilm-finetuning/output/Qwen-1_8B-Chat_on_giaothong_rag/', device_map=0, do_update_lm_head=True, echo=True)
llm72 = models.get_qwen_guidance('/public-llm/Qwen-14B-Chat-Int4/',device_map=0, do_update_lm_head=True, echo=True)

2023-12-30 17:36:50.671 | INFO     | guidance.models.transformers._qwen:get_qwen_guidance:227 - Load Qwen from "/public-llm/Qwen-14B-Chat-Int4/" with device_map "0"
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

2023-12-30 17:37:13.438 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


len(tokenizer)=151851


## Search

In [16]:
import requests

def search(query, k=3, port=19001):
    url = f"http://localhost:{port}/api/search"
    params = {
        "query": query,
        "return_dict": "true",
        'k': k
    }
    response = requests.get(url, params=params, proxies={'http':'', 'https':''})
    topk = response.json()['topk']
    # print(topk[0].keys())
    # docid_to_text = {t['pid']:f'{t["title"]}| {t["text"]}' for t in topk}
    docid_to_text = [{'doc_id': t['pid'], 'text': f'{t["title"]}| {t["text"]}'} for t in topk]
    return docid_to_text
    
query = 'Thủ tục đăng ký biển số xe máy mới nhất'

search_results = search(query, k=3)

print(json.dumps(search_results, ensure_ascii=False, indent=2))

[
  {
    "doc_id": 30668,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tục đăng ký sang tên, di chuyển xe\n1. Thủ tục thu hồi\na) Chủ xe kê khai giấy khai thu hồi đăng ký, biển số xe trên cổng dịch vụ công; cung cấp mã hồ sơ đăng ký xe trực tuyến; nộp hồ sơ thu hồi quy định tại khoản 1 Điều 14 Thông tư này và nhận giấy hẹn trả kết quả đăng ký xe theo quy định;\nb) Sau khi kiểm tra hồ sơ xe hợp lệ, cơ quan đăng ký xe cấp chứng nhận thu hồi đăng ký, biển số xe theo quy định (có dán bản chà số máy, số khung và đóng dấu giáp lai của cơ quan đăng ký xe trên bản chà số máy, số khung xe): 01 bản trả cho chủ xe; 01 bản lưu hồ sơ xe; trường hợp mất chứng nhận đăng ký xe thì thực hiện xác minh theo quy định."
  },
  {
    "doc_id": 121823,
    "text": "Thông tư 24/2023/TT-BCA cấp thu hồi đăng ký biển số xe cơ giới mới nhất\nChương II\nMục 2. ĐĂNG KÝ SANG TÊN, DI CHUYỂN XE |  Điều 15. Thủ tụ

## Guidance

In [6]:
import guidance

In [7]:
@guidance
def giaothong_rag(lm, query, gt_answer=None, k=None, db="qa"):
    port = 19003 if db == "qa" else 19001
    if k is None:
        # elase randomly choose int form 5 to 10
        k = 3 if db == "qa" else np.random.randint(3, 7)
    relevant_docs = search(query, k=k, port=port)
    ks = [d['doc_id'] for d in relevant_docs]
    with system():
        lm = (
            lm
            + "You are now tasked to answer user questions based on the retrieved documents\nResponse language: Vietnamese"
        )

    with user():
        lm += """Query: {}
        Retrieved documents: {}
        Guide:
            - Hãy phân tích sau đó đưa ra kết luận về câu hỏi này
            - Response language: Vietnamese
        """.format(
            query, json.dumps(relevant_docs, ensure_ascii=False, indent=2)
        )

    with assistant():
        lm += f"**Câu hỏi** {query}"
        lm += "\n\n**Phân tích**" + gen(max_tokens=2000, name="reasoning", stop="**")
        lm += "**Kết luận**" + gen(
            max_tokens=2000, name="conclusion", stop='\n\n'
        )
        lm += "\n\n Tham khảo:" + gen() + '. [doc_id: '+ select(ks)+']'
    # with user():
    #     lm += "Hãy trích dẫn nguồn tham khảo cho kết luận trên, nếu không có hãy trả lời `Không có tài liệu liên quan`, nếu có hãy trả lời `Các tài liệu liên quan là ...`\n\
    #     Khi bạn trích dẫn nguồn, hãy liệt kê theo thứ tự ưu tiên. Tài liệu quan trọng nhất nên được đặt ở vị trí đầu tiên. \n\
    #     Cách định dạng: ```reference[{'doc_id': 'document_id', 'point': điểm liên quan)}, ...]``` \n\
    #     Điểm liên quan được tính trên thang điểm 10 với 10 là điểm liên quan nhất, 0 là không liên quan."
    # with assistant():
    #     lm += select(['Không có tài liệu liên quan', 'Các tài liệu liên quan'], name="has_reference")
    #     if lm['has_reference'] == 'Các tài liệu liên quan':
    #         lm += "\n ```reference["
    #         lm += gen(max_tokens=2000, name="reference", stop="]```")
    #         lm += "]```"
    return lm



In [8]:
# q = 'Mức phạt tiền nồng độ cồn đối với xe máy'
# lm = llm+giaothong_rag(query = q, db='docs', k=3)

In [9]:
# docid2text = pd.DataFrame(search_results).set_index('doc_id')['text'].to_dict()

In [10]:
# docid2text.k

In [11]:
# data = load_by_ext('../../data/chat-formated-dataset/tvpl_giaothong_rag_conversation_from_qwen72b.json')

In [12]:
# user = data[0]['conversations'][0]['value']

# QAC

In [30]:
@guidance
def giaothong_NLI(lm, query, gt_answer, k=1, db="docs"):
    port = 19003 if db == "qa" else 19001
    if k is None:
        # elase randomly choose int form 5 to 10
        k = 3 if db == "qa" else np.random.randint(3, 7)
    relevant_docs = search(query, k=10, port=port)
    retrived_doc = np.random.choice(relevant_docs)
    retrived_doc = retrived_doc['text']
    ks = [d['doc_id'] for d in relevant_docs]
    with system():
        lm = (
            lm +\
    '''
    Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:
    
    Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
    Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
    Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
    Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.
    '''
        )
    with user():
        lm += f'''
        Câu hỏi: ```{query}```
        
        Câu trả lời mong đợi: ```{gt_answer}```
        
        Tài liệu đã truy xuất: ```{retrived_doc}```
                
        Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.
    
        '''
    with assistant():
        nl = '\n'
        lm += f'''
        Để xác định liệu tài liệu đã truy xuất có hỗ trợ cho câu hỏi và câu trả lời hay không, chúng ta sẽ:
    
        1. Phân Tích Bối Cảnh Tài Liệu: Đánh giá nội dung tổng quan và ngữ cảnh của tài liệu.
        2. So Sánh Câu Hỏi và Tài Liệu: Kiểm tra sự tham chiếu trực tiếp hoặc các khái niệm liên quan trong tài liệu phù hợp với câu hỏi.
        3. Liên Hệ Câu Trả Lời và Tài Liệu: Xác định xem các yếu tố chính hoặc chi tiết cụ thể của câu trả lời có được hỗ trợ bởi tài liệu không.
        4. Đánh Giá Mâu Thuẫn: Xác định bất kỳ mâu thuẫn nào giữa tài liệu và câu trả lời mong đợi.
        5. Rút Ra Kết Luận: Dựa trên phân tích, kết luận liệu tài liệu có hỗ trợ hay mâu thuẫn với câu trả lời mong đợi.
    
        Hãy tiếp tục với việc đánh giá:
    
        - Phân Tích Bối Cảnh Tài Liệu: 
        Tài liệu chủ yếu thảo luận về {gen(stop=nl)} 
    
        - So Sánh Câu Hỏi và Tài Liệu:
        Xét về câu hỏi, tài liệu {gen(stop=nl)} 
    
        - Liên Hệ Câu Trả Lời và Tài Liệu:
        Liên quan đến câu trả lời mong đợi, tài liệu {gen(stop=nl)} 
    
        - Đánh Giá Mâu Thuẫn:
        Bất kỳ mâu thuẫn nào được tìm thấy là {gen(stop=nl)} 
    
        - Kết Luận Cuối Cùng:
        Dựa trên phân tích, tài liệu {gen(stop=nl)}
    
        '''

    with user():
        lm += '''Now make a conclusion base on the following definition:
        - Direct Support: Seek document statements or data affirming the answer.
        - Indirect Support: Check for background or context supporting the answer.
        - Contradiction: Identify any document content contradicting the answer.
        - Neutral/Irrelevant: Note information unrelated to the question and answer.'''

    with assistant():
        lm += 'Based on the analysis, the conclusion is as follows: '+gen()
    with user():
        lm += 'Please select one of the following options: \n - 1. Direct Support \n - 2. Indirect Support \n - 3. Contradiction \n - 4. Neutral/Irrelevant'
    with assistant():
        choices = ['1. Direct Support', '2. Indirect Support', '3. Contradiction', '4. Neutral/Irrelevant']
        lm += select(choices)
        
    return lm

In [31]:
qa = questions[1]

In [32]:
lm = llm72+giaothong_NLI(qa['question'], qa['short_answer'])

KeyboardInterrupt: 

In [21]:
lm._current_prompt()

"<|im_start|>system\n\n\nYou have been assigned the Natural Language Inference (NLI) task, which involves analyzing a question, a given answer, and a retrieved document. Follow these steps:\n\nUnderstand the Question: Grasp the full context and specifics.\nAnalyze the Answer: Examine its key points, arguments, and details.\nReview the Document: Look for information relevant to the question and answer.\n\n<|im_end|>\n<|im_start|>user\n\nCâu hỏi: Điều kiện thi lái xe tải (hạng C) hoặc xe 16 chỗ (hạng D) bao gồm những gì?\nĐáp án: Điều kiện thi lái xe tải (hạng C) hoặc xe 16 chỗ (hạng D) bao gồm độ tuổi và sức khỏe theo quy định của pháp luật.\n\nRetrived document: [{'doc_id': 10014, 'text': 'Luật giao thông đường bộ 2008 23/2008/QH12 mới nhất\\nCHƯƠNG V |  Điều 60. Tuổi, sức khỏe của người lái xe\\n1. Độ tuổi của người lái xe quy định như sau:\\na) Người đủ 16 tuổi trở lên được lái xe gắn máy có dung tích xi-lanh dưới 50 cm 3 ;\\nb) Người đủ 18 tuổi trở lên được lái xe mô tô hai bánh, xe